# Bottleneck Features Extraction

---

Before supplying an image to a pre-trained network in Keras, there are some required preprocessing steps. When using TensorFlow as backend,  Keras neural networks takes a 4D-array as input, with shape `(nb_images, rows, columns, channels)`, where `nb_images` corresponds to the total number of images, and rows, columns, and channels correspond to the number of `rows`, `columns`, and `channels` for each image, respectively.

The idea is to compare the performances of various deep learning models on the same dataset and select the most promising one for this project. For this reason, only a small fraction of the data are considered to produce the bottleneck features. The models are compared in this [notebook](compare.ipynb).

## 1. Load Files and Targets
The images from the training, validation and test datasets are first loaded.

In [2]:
import numpy as np
from common import *

train_ids, train_targets = load_dataset('data/preprocess/train.npz')
valid_ids, valid_targets = load_dataset('data/preprocess/valid.npz')
test_ids, test_targets = load_dataset('data/preprocess/test.npz')

train_files = np.array(['data/train_photos/' + str(i) + '.jpg' for i in train_ids])
valid_files = np.array(['data/train_photos/' + str(i) + '.jpg' for i in valid_ids])
test_files = np.array(['data/train_photos/' + str(i) + '.jpg' for i in test_ids])

print('There are %d images in the training dataset' % len(train_files))
print('There are %d images in the validation dataset' % len(valid_files))
print('There are %d images in the test dataset' % len(test_files))

There are 176131 images in the training dataset
There are 29355 images in the validation dataset
There are 29356 images in the test dataset


In [3]:
nb_train, nb_valid, nb_test = (20000,2000,2000)

## 2. Bottleneck Features

### A. VGG16

#### - $\alpha$ Preprocess Tensors
The `paths_to_tensor` function used in the cell below takes a numpy array of string-valued image paths as input and returns a 4D-array with shape (`nb_images`, 224, 224, 3).

In addition, getting the 4D tensor ready for any pre-trained model in Keras, requires some additional processing. First, the RGB image is converted to BGR by reordering the channels. All pre-trained models have the additional normalization step that the mean pixel (expressed in RGB as [103.939, 116.779, 123.68] and calculated from all pixels in all images in ImageNet) must be subtracted from every pixel in each image. This is implemented in the imported function `preprocess_input`.

In [4]:
from keras.applications.vgg16 import preprocess_input

train_tensors = preprocess_input(paths_to_tensor(train_files[:nb_train]))
valid_tensors = preprocess_input(paths_to_tensor(valid_files[:nb_valid]))
test_tensors = preprocess_input(paths_to_tensor(test_files[:nb_test]))

100%|██████████| 2000/2000 [00:05<00:00, 334.44it/s]


#### - $\beta$ Import the VGG16 Model, with the Final Fully-Connected Layers Removed
We remove the final layers of the pre-trained network to obtain the bottleneck features.

In [5]:
from keras.applications.vgg16 import VGG16

model = VGG16(include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

#### - $\gamma$ Extract Output of Final Max Pooling Layer
Now, the network stored in `model` is a truncated version of the VGG16 network, where the final three fully-connected layers have been removed.  In this case, `model.predict` returns a 3D array (with dimensions 7 x 7 x 512) corresponding to the final max pooling layer of VGG16.  The dimensionality of the obtained output from passing the 4D-array through the model is `(nb_images, 7, 7, 512)`.

In [6]:
train_vgg16 = model.predict(train_tensors)
valid_vgg16 = model.predict(valid_tensors)
test_vgg16  = model.predict(test_tensors)

In [7]:
print(train_vgg16.shape)
print(valid_vgg16.shape)
print(test_vgg16.shape)

(20000, 7, 7, 512)
(2000, 7, 7, 512)
(2000, 7, 7, 512)


In [8]:
np.savez('data/bottleneck_features/yelp_vgg16.npz', 
         train_features=train_vgg16, train_targets=train_targets[:nb_train],
         valid_features=valid_vgg16, valid_targets=valid_targets[:nb_valid],
         test_features=test_vgg16, test_targets=test_targets[:nb_test])

### B. Xception

#### - $\alpha$ Preprocess Tensors

In [9]:
from keras.applications.xception import preprocess_input

train_tensors = preprocess_input(paths_to_tensor(train_files[:nb_train]))
valid_tensors = preprocess_input(paths_to_tensor(valid_files[:nb_valid]))
test_tensors = preprocess_input(paths_to_tensor(test_files[:nb_test]))

100%|██████████| 2000/2000 [00:06<00:00, 333.11it/s]


#### - $\beta$ Import the Xception Model, with the Final Fully-Connected Layers Removed

In [10]:
from keras.applications.xception import Xception

model = Xception(include_top=False)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

#### - $\gamma$ Extract Output of Final Activation Layer

In [11]:
train_xception = model.predict(train_tensors)
valid_xception = model.predict(valid_tensors)
test_xception  = model.predict(test_tensors)

In [12]:
print(train_xception.shape)
print(valid_xception.shape)
print(test_xception.shape)

(20000, 7, 7, 2048)
(2000, 7, 7, 2048)
(2000, 7, 7, 2048)


In [13]:
np.savez('data/bottleneck_features/yelp_xception.npz', 
         train_features=train_xception, train_targets=train_targets[:nb_train],
         valid_features=valid_xception, valid_targets=valid_targets[:nb_valid],
         test_features=test_xception, test_targets=test_targets[:nb_test])

### C. ResNet50

#### - $\alpha$ Preprocess Tensors

In [14]:
from keras.applications.resnet50 import preprocess_input

train_tensors = preprocess_input(paths_to_tensor(train_files[:nb_train]))
valid_tensors = preprocess_input(paths_to_tensor(valid_files[:nb_valid]))
test_tensors = preprocess_input(paths_to_tensor(test_files[:nb_test]))

100%|██████████| 2000/2000 [00:05<00:00, 334.62it/s]


#### - $\beta$ Import the ResNet50 Model, with the Final Fully-Connected Layers Removed

In [15]:
from keras.applications.resnet50 import ResNet50

model = ResNet50(include_top=False)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_3[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

#### - $\gamma$ Extract Output of Final Average Pooling Layer

In [16]:
train_resnet50 = model.predict(train_tensors)
valid_resnet50 = model.predict(valid_tensors)
test_resnet50  = model.predict(test_tensors)

In [17]:
print(train_resnet50.shape)
print(valid_resnet50.shape)
print(test_resnet50.shape)

(20000, 1, 1, 2048)
(2000, 1, 1, 2048)
(2000, 1, 1, 2048)


In [18]:
np.savez('data/bottleneck_features/yelp_resnet50.npz', 
         train_features=train_resnet50, train_targets=train_targets[:nb_train],
         valid_features=valid_resnet50, valid_targets=valid_targets[:nb_valid],
         test_features=test_resnet50, test_targets=test_targets[:nb_test])

### D. InceptionV3

#### - $\alpha$ Preprocess Tensors

In [19]:
from keras.applications.inception_v3 import preprocess_input

train_tensors = preprocess_input(paths_to_tensor(train_files[:nb_train]))
valid_tensors = preprocess_input(paths_to_tensor(valid_files[:nb_valid]))
test_tensors = preprocess_input(paths_to_tensor(test_files[:nb_test]))

100%|██████████| 2000/2000 [00:06<00:00, 331.10it/s]


#### - $\beta$ Import the Inception Model, with the Final Fully-Connected Layers Removed

In [20]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(include_top=False)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

#### - $\gamma$ Extract Output of Final Concatenate Layer

In [21]:
train_inception = model.predict(train_tensors)
valid_inception = model.predict(valid_tensors)
test_inception  = model.predict(test_tensors)

In [22]:
print(train_inception.shape)
print(valid_inception.shape)
print(test_inception.shape)

(20000, 5, 5, 2048)
(2000, 5, 5, 2048)
(2000, 5, 5, 2048)


In [23]:
np.savez('data/bottleneck_features/yelp_inception.npz', 
         train_features=train_inception, train_targets=train_targets[:nb_train],
         valid_features=valid_inception, valid_targets=valid_targets[:nb_valid],
         test_features=test_inception, test_targets=test_targets[:nb_test])